In [ ]:
# Backup of misc scripts

# Generate preview thumbnails as jpgs

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

# === Параметры ===
SOURCE_FOLDER = "images"
OUTPUT_FOLDER = "thumbnails"
SIZE = (64, 64)

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

images = [f for f in os.listdir(SOURCE_FOLDER) if f.lower().endswith(".jpg")]

for fname in tqdm(images, desc="📷 Создание превью", ncols=80):
    in_path = os.path.join(SOURCE_FOLDER, fname)
    out_path = os.path.join(OUTPUT_FOLDER, fname)
    try:
        img = Image.open(in_path)
        img.thumbnail(SIZE)
        img = img.convert("RGB")  # на всякий случай
        img.save(out_path, "JPEG")
    except Exception as e:
        print(f"❌ {fname}: {e}")

print(f"✅ Сжато {len(images)} изображений в '{OUTPUT_FOLDER}'")

In [ ]:
import os
import zipfile
import pandas as pd
from tqdm import tqdm
from xml.sax.saxutils import escape

# === Константы ===
ROOT = os.getcwd()
CSV_PATH = os.path.join(ROOT, "csv", "photos.csv")
KMZ_FOLDER = os.path.join(ROOT, "kml")
FULL_IMAGE_BASE_URL = "https://mloktionov.github.io/PhotoMaps/images/"
THUMBNAIL_BASE_URL = "https://mloktionov.github.io/PhotoMaps/thumbnails/"

os.makedirs(KMZ_FOLDER, exist_ok=True)

# Загрузка CSV
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=["latitude", "longitude"])

# Группировка по годам
years = sorted(df["year"].dropna().unique().astype(int))

for year in years:
    df_year = df[df["year"] == year]

    kml_parts = [f'''<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Document>
    <name>PhotoMaps {year}</name>
''']

    for _, row in tqdm(df_year.iterrows(), total=len(df_year), desc=f"📍 {year}", ncols=80):
        fname = row["filename"]
        lat = row["latitude"]
        lon = row["longitude"]
        month = int(row["month"])
        day = int(row["day"])
        date_label = f"{month:02d}-{day:02d}"

        image_url = FULL_IMAGE_BASE_URL + fname
        thumb_url = THUMBNAIL_BASE_URL + fname

        placemark = f"""
        <Placemark>
          <name>{date_label}</name>
          <Style>
            <IconStyle>
              <scale>1.2</scale>
              <Icon>
                <href>{thumb_url}</href>
              </Icon>
            </IconStyle>
          </Style>
          <description><![CDATA[
            <div style="font-family: sans-serif; font-size: 13px;">
              <img src="{image_url}" width="600"><br>
              <i>{escape(fname)}</i>
            </div>
          ]]></description>
          <Point>
            <coordinates>{lon},{lat},0</coordinates>
          </Point>
        </Placemark>
        """
        kml_parts.append(placemark)

    kml_parts.append("  </Document>\n</kml>")

    # Сохраняем doc.kml
    doc_kml_path = os.path.join(KMZ_FOLDER, f"doc_{year}.kml")
    with open(doc_kml_path, "w", encoding="utf-8") as f:
        f.write("\n".join(kml_parts))

    # Упаковываем doc.kml в KMZ
    kmz_path = os.path.join(KMZ_FOLDER, f"PhotoMaps_{year}.kmz")
    with zipfile.ZipFile(kmz_path, "w", zipfile.ZIP_DEFLATED) as kmz:
        kmz.write(doc_kml_path, arcname="doc.kml")

    print(f"✅ Готово: {kmz_path}")

In [ ]:
# ---- Create Thumbnails

# generate kmz with jpg preview

In [ ]:
import os
import zipfile
import pandas as pd
from tqdm import tqdm
from xml.sax.saxutils import escape

# === Константы ===
ROOT = os.getcwd()
CSV_PATH = os.path.join(ROOT, "csv", "photos.csv")
THUMBNAIL_FOLDER = os.path.join(ROOT, "thumbnails")
KMZ_FOLDER = os.path.join(ROOT, "kml")
FULL_IMAGE_BASE_URL = "https://mloktionov.github.io/PhotoMaps/images/"
THUMBNAIL_RELATIVE_PATH = "thumbnails"

# === Подготовка
os.makedirs(KMZ_FOLDER, exist_ok=True)
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=["latitude", "longitude"])

# === Разбивка по годам
years = sorted(df['year'].dropna().unique().astype(int))

for year in years:
    df_year = df[df['year'] == year]

    kml_parts = [f'''<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Document>
    <name>PhotoMaps {year}</name>
''']

    for _, row in tqdm(df_year.iterrows(), total=len(df_year), desc=f"📍 {year}", ncols=80):
        fname = row["filename"]
        lat = row["latitude"]
        lon = row["longitude"]
        month = int(row["month"])
        day = int(row["day"])
        date_label = f"{month:02d}-{day:02d}"

        thumb_name = fname.replace(".jpg", ".png").replace(".JPG", ".png")
        image_url = FULL_IMAGE_BASE_URL + fname
        icon_href = os.path.join(THUMBNAIL_RELATIVE_PATH, thumb_name)

        placemark = f"""
        <Placemark>
          <name>{date_label}</name>
          <Style>
            <IconStyle>
              <scale>1.2</scale>
              <Icon>
                <href>{icon_href}</href>
              </Icon>
            </IconStyle>
          </Style>
          <description><![CDATA[
            <div style="font-family: sans-serif; font-size: 13px;">
              <img src="{image_url}" width="600"><br>
              <i>{escape(fname)}</i>
            </div>
          ]]></description>
          <Point>
            <coordinates>{lon},{lat},0</coordinates>
          </Point>
        </Placemark>
        """
        kml_parts.append(placemark)

    kml_parts.append("  </Document>\n</kml>")

    # Сохраняем doc.kml
    doc_kml_path = os.path.join(KMZ_FOLDER, f"doc_{year}.kml")
    with open(doc_kml_path, "w", encoding="utf-8") as f:
        f.write("\n".join(kml_parts))

    # Упаковка в KMZ
    kmz_path = os.path.join(KMZ_FOLDER, f"PhotoMaps_{year}.kmz")
    with zipfile.ZipFile(kmz_path, "w", zipfile.ZIP_DEFLATED) as kmz:
        kmz.write(doc_kml_path, arcname="doc.kml")
        needed_thumbs = df_year["filename"].str.replace(".jpg", ".png").str.replace(".JPG", ".png").tolist()
        for fname in os.listdir(THUMBNAIL_FOLDER):
            if fname in needed_thumbs:
                full_path = os.path.join(THUMBNAIL_FOLDER, fname)
                arcname = os.path.join("thumbnails", fname)
                kmz.write(full_path, arcname=arcname)

    print(f"✅ Сохранено: {kmz_path}")

In [ ]:
import os
import pandas as pd
from xml.dom.minidom import Document
from tqdm import tqdm

# === 🔧 КОНСТАНТЫ ===
ROOT = os.getcwd()
CSV_PATH = os.path.join(ROOT, "csv", "photos.csv")
THUMB_URL_BASE = "https://mloktionov.github.io/PhotoMaps/thumbnails"
IMAGE_URL_BASE = "https://mloktionov.github.io/PhotoMaps/images"
OUTPUT_FOLDER = os.path.join(ROOT, "kml", "mymaps_export")

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# === 📄 Загрузка CSV ===
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=["latitude", "longitude", "year"])
df["year"] = df["year"].astype(int)

# === 🗂️ Группировка по годам ===
for year, group in tqdm(df.groupby("year"), desc="📦 Генерация файлов по годам"):
    doc = Document()
    kml = doc.createElement("kml")
    kml.setAttribute("xmlns", "http://www.opengis.net/kml/2.2")
    doc.appendChild(kml)

    document = doc.createElement("Document")
    kml.appendChild(document)

    for _, row in group.iterrows():
        placemark = doc.createElement("Placemark")

        # 🏷 Название
        name = doc.createElement("name")
        try:
            month_day = row["filename"][4:12]
            name.appendChild(doc.createTextNode(f"{month_day[4:6]}-{month_day[6:]}"))
        except:
            name.appendChild(doc.createTextNode(row["filename"]))
        placemark.appendChild(name)

        # 🔗 Описание (ссылка на полное изображение)
        desc = doc.createElement("description")
        img_url = f"{IMAGE_URL_BASE}/{row['filename']}"
        desc.appendChild(doc.createTextNode(f"📷 {img_url}"))
        placemark.appendChild(desc)

        # 📍 Геопозиция
        point = doc.createElement("Point")
        coords = doc.createElement("coordinates")
        coords.appendChild(doc.createTextNode(f"{row['longitude']},{row['latitude']},0"))
        point.appendChild(coords)
        placemark.appendChild(point)

        document.appendChild(placemark)

    # 💾 Сохраняем файл
    out_path = os.path.join(OUTPUT_FOLDER, f"mymaps_{year}.kml")
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(doc.toprettyxml(indent="  "))

print(f"✅ Готово: KML-файлы для My Maps в {OUTPUT_FOLDER}")

In [ ]:
# Generate KML files for Google My Maps

import os
import pandas as pd
from xml.sax.saxutils import escape
from tqdm import tqdm

# === 🔧 Константы ===
ROOT = os.getcwd()
CSV_PATH = os.path.join(ROOT, "csv", "photos.csv")
THUMBNAILS_URL = "https://mloktionov.github.io/PhotoMaps/thumbnails"
KML_OUTPUT_DIR = os.path.join(ROOT, "kml")

# 📁 Убедимся, что папка есть
os.makedirs(KML_OUTPUT_DIR, exist_ok=True)

# 📄 Загрузка данных
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=["latitude", "longitude"])

# Группировка по годам
groups = df.groupby("year")

for year, group in groups:
    kml_name = f"doc_{year}.kml"
    kml_path = os.path.join(KML_OUTPUT_DIR, kml_name)

    placemarks = []
    for _, row in tqdm(group.iterrows(), total=len(group), desc=f"📍 {year}", ncols=80):
        fname = row["filename"]
        lat = row["latitude"]
        lon = row["longitude"]
        month = str(row["month"]).zfill(2)
        day = str(row["day"]).zfill(2)
        label = f"{month}-{day}"
        thumb_url = f"{THUMBNAILS_URL}/{os.path.splitext(fname)[0]}.png"
        full_url = f"https://mloktionov.github.io/PhotoMaps/images/{fname}"

        placemark = f"""
<Placemark>
  <name>{escape(label)}</name>
  <Style>
    <IconStyle>
      <scale>1.0</scale>
      <Icon>
        <href>{thumb_url}</href>
      </Icon>
    </IconStyle>
  </Style>
  <description><![CDATA[
    <img src="{thumb_url}" width="128"/><br/>
    <a href="{full_url}">{fname}</a>
  ]]></description>
  <Point>
    <coordinates>{lon},{lat},0</coordinates>
  </Point>
</Placemark>
"""
        placemarks.append(placemark.strip())

    # 📦 Обёртка в документ
    kml_content = f"""<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
<Document>
  <name>PhotoMap {year}</name>
  {''.join(placemarks)}
</Document>
</kml>
"""

    # 💾 Сохраняем
    with open(kml_path, "w", encoding="utf-8") as f:
        f.write(kml_content)

print(f"\n✅ Сгенерированы KML-файлы по годам в '{KML_OUTPUT_DIR}'")

In [ ]:
---- manipulating csv after editing, not supported anymore

In [ ]:
import pandas as pd
import os

# === 📁 Пути
ROOT = os.path.abspath(".")
CSV_FOLDER = os.path.join(ROOT, "csv")

PHOTOS_CSV = os.path.join(CSV_FOLDER, "photos.csv")
MISSING_CSV = os.path.join(CSV_FOLDER, "missing_coords.csv")
UPDATED_COORDS_1 = os.path.join(CSV_FOLDER, "coords_updated.csv")
UPDATED_COORDS_2 = os.path.join(CSV_FOLDER, "updated_coords_from_google_photos.csv")

# === 📦 Загрузка
photos_df = pd.read_csv(PHOTOS_CSV)
missing_df = pd.read_csv(MISSING_CSV)
coords_df_1 = pd.read_csv(UPDATED_COORDS_1)
coords_df_2 = pd.read_csv(UPDATED_COORDS_2)

# === 🔗 Объединяем оба CSV
coords_all = pd.concat([coords_df_1, coords_df_2], ignore_index=True)

# === 🧭 Фильтруем только записи с координатами
coords_all = coords_all.dropna(subset=["latitude", "longitude"])

# === 🕓 Добавляем дату из missing_coords.csv
coords_all = pd.merge(coords_all, missing_df[["filename", "year", "month", "day"]], on="filename", how="left", suffixes=("", "_drop"))
coords_all = coords_all[[c for c in coords_all.columns if not c.endswith("_drop")]]
coords_all["folder"] = "photos"

# === 📝 Обновляем photos.csv
photos_df = pd.concat([photos_df, coords_all], ignore_index=True)
photos_df.to_csv(PHOTOS_CSV, index=False)
print(f"✅ Обновлён: {PHOTOS_CSV} ({len(photos_df)} записей)")

# === 🧹 Удаляем из missing_coords
remaining_missing = missing_df[~missing_df["filename"].isin(coords_all["filename"])]
remaining_missing.to_csv(MISSING_CSV, index=False)
print(f"🧹 Обновлён: {MISSING_CSV} ({len(remaining_missing)} осталось)")

# === 🧽 Чистим временные файлы
coords_all.to_csv(UPDATED_COORDS_1, index=False)
if os.path.exists(UPDATED_COORDS_2):
    os.remove(UPDATED_COORDS_2)
    print("🗑️ Удалён: updated_coords_from_google_photos.csv")